In [1]:
import re
import json

from simple_websocket_server import WebSocketServer, WebSocket

class Controller: 
    def process(self, telemetry):
        print(telemetry)
        return {}
    

class SimulatorClient(WebSocket):
    incoming_pattern = re.compile('42\["telemetry",\s*(.*)\s*\]')
    response_template = '42["manual",%s]'
    
    def connected(self):
        self.controller = Controller()
        
    def handle(self):
        command = self._receive_telemetry(self.data)
        self._send_command(command)
        
    def _receive_telemetry(self, package):
        match_data = self.incoming_pattern.match(package)
        if not match_data: return None
        
        telemetry = json.loads(match_data.group(1))
        return self.controller.process(telemetry)
            
    def _send_command(self, command):
        response = self.response_template % json.dumps(command)
        self.send_message(response)
        

class UdacitySimulator: 
    def run(self):
        server = WebSocketServer('', 4567, SimulatorClient)
        try:
            server.serve_forever()
        except KeyboardInterrupt: 
            server.close()


In [2]:
sim = UdacitySimulator()
sim.run()

{'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x': -40.62008, 'y': 108.7301, 'steering_angle': 0, 'throttle': 0, 'speed': 1.218817e-06}
{'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x': -40.62008, 'y': 108.7301, 'steering_angle': 0, 'throttle': 0, 'speed': 1.376143e-06}
{'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x': -40.62008, 'y': 108.7301, 'steering_angle': 0, 'throttle': 0, 'speed': 1.376143e-06}
{'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x

{'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x': -40.62008, 'y': 108.7301, 'steering_angle': 0, 'throttle': 0, 'speed': 9.216697e-06}
{'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x': -40.62008, 'y': 108.7301, 'steering_angle': 0, 'throttle': 0, 'speed': 2.598458e-06}
{'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x': -40.62008, 'y': 108.7301, 'steering_angle': 0, 'throttle': 0, 'speed': 2.598458e-06}
{'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x

{'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x': -40.62008, 'y': 108.7301, 'steering_angle': 0, 'throttle': 0, 'speed': 6.122007e-06}
{'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x': -40.62008, 'y': 108.7301, 'steering_angle': 0, 'throttle': 0, 'speed': 2.866993e-06}
{'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x': -40.62008, 'y': 108.7301, 'steering_angle': 0, 'throttle': 0, 'speed': 1.988156e-06}
{'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x

{'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x': -40.62008, 'y': 108.7301, 'steering_angle': 0, 'throttle': 0, 'speed': 6.909984e-08}
{'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x': -40.62008, 'y': 108.7301, 'steering_angle': 0, 'throttle': 0, 'speed': 4.008804e-07}
{'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x': -40.62008, 'y': 108.7301, 'steering_angle': 0, 'throttle': 0, 'speed': 6.333365e-07}
{'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x

{'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x': -40.62008, 'y': 108.7301, 'steering_angle': 0, 'throttle': 0, 'speed': 9.484256e-07}
{'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x': -40.62008, 'y': 108.7301, 'steering_angle': 0, 'throttle': 0, 'speed': 2.738885e-06}
{'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x': -40.62008, 'y': 108.7301, 'steering_angle': 0, 'throttle': 0, 'speed': 2.738885e-06}
{'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x

...

{'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x': -40.62008, 'y': 108.7301, 'steering_angle': 0, 'throttle': 0, 'speed': 4.137282e-06}
{'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x': -40.62008, 'y': 108.7301, 'steering_angle': 0, 'throttle': 0, 'speed': 4.137282e-06}
{'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x': -40.62008, 'y': 108.7301, 'steering_angle': 0, 'throttle': 0, 'speed': 6.858569e-06}
{'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x

{'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x': -40.62008, 'y': 108.7301, 'steering_angle': 0, 'throttle': 0, 'speed': 9.112815e-06}
{'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x': -40.62008, 'y': 108.7301, 'steering_angle': 0, 'throttle': 0, 'speed': 1.197326e-05}
{'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x': -40.62008, 'y': 108.7301, 'steering_angle': 0, 'throttle': 0, 'speed': 1.463436e-05}
{'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x

{'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x': -40.62008, 'y': 108.7301, 'steering_angle': 0, 'throttle': 0, 'speed': 9.878426e-06}
{'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x': -40.62008, 'y': 108.7301, 'steering_angle': 0, 'throttle': 0, 'speed': 6.624086e-06}
{'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x': -40.62008, 'y': 108.7301, 'steering_angle': 0, 'throttle': 0, 'speed': 6.624086e-06}
{'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x

{'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x': -40.62008, 'y': 108.7301, 'steering_angle': 0, 'throttle': 0, 'speed': 1.207384e-05}
{'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x': -40.62008, 'y': 108.7301, 'steering_angle': 0, 'throttle': 0, 'speed': 1.207384e-05}
{'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x': -40.62008, 'y': 108.7301, 'steering_angle': 0, 'throttle': 0, 'speed': 1.00124e-05}
{'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x'

{'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x': -40.62008, 'y': 108.7301, 'steering_angle': 0, 'throttle': 0, 'speed': 4.330344e-06}
{'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x': -40.62008, 'y': 108.7301, 'steering_angle': 0, 'throttle': 0, 'speed': 3.297001e-06}
{'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x': -40.62008, 'y': 108.7301, 'steering_angle': 0, 'throttle': 0, 'speed': 3.297001e-06}
{'ptsx': [-32.16173, -43.49173, -61.09, -78.29172, -93.05002, -107.7717], 'ptsy': [113.361, 105.941, 92.88499, 78.73102, 65.34102, 50.57938], 'psi_unity': 4.120315, 'psi': 3.733667, 'x